# Context Managers and the with Statement

## Example

### Open a File

In [1]:
with open('1_assert_in_python.py') as f:
    pass

### Thread Lock

In [2]:
import threading

In [3]:
some_lock = threading.Lock()

In [4]:
with some_lock:
    pass

## Support With in Your Own Objects

### Simple Protocol

How: by implementing the so-called context managers.

Q: What’s a context manager? 

A: It’s a simple “protocol” (or interface) that your object needs to follow in order to support the with statement. 

in a word, add __enter__ and __exit__ methods to an object.


In [5]:
class ManagedFile:
    def __init__(self, name):
        self.name = name
        
    def __enter__(self):
        self.file = open(self.name, 'w')
        return self.file
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        if self.file:
            self.file.close()

In [6]:
with ManagedFile('hello.txt') as f:
    f.write('hello, world!')
    f.write('bye now')

### Decorator from Standard

Writing a class-based context manager isn’t the only way to support the with statement in Python.

The contextlib utility module in the standard library provides a few more abstractions built on top of the basic context manager protocol. 

In [7]:
from contextlib import contextmanager

In [8]:
@contextmanager
def managed_file(name):
    try:
        f = open(name, 'w')
        yield f
    finally:
        f.close()

In [9]:
with managed_file('hello.txt') as f:
    f.write('hello, world!')
    f.write('bye now')

## Exercise

### Indentation

How to implement this?

```python
with Indenter() as indent:
    indent.print('hi!')
    with indent:
        indent.print('hello')
        with indent:
            indent.print('bonjour')
    indent.print('hey') 
```

In [20]:
class Indenter:
    def __init__(self):
        self.indent = ''
        
    def __enter__(self):
        self.indent += 4 * ' '
        return self
        
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.indent = self.indent[:-4]
        
    def print(self, s):
        print(self.indent + s)

In [25]:
with Indenter() as indent:
    indent.print('hi!')
    with indent:
        indent.print('hello')
        with indent:
            indent.print('bonjour')
    indent.print('hey')

    hi!
        hello
            bonjour
    hey


a reference:

```python
class Indenter:
    def __init__(self):
        self.level = 0
        
    def __enter__(self):
        self.level += 1
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.level -= 1
        
    def print(self, s):
        print(' ' * 4 * self.level + s)
```

### Execution Time Calculator

In [27]:
import time

In [48]:
# class based implementation
class ExecutionTimeCalculator:
    def __init__(self, func, *args, **kwargs):
        self.func = func
        self.args = args
        self.kwargs = kwargs
    
    def __enter__(self):
        self.start = time.time()
        self.func(*self.args, **self.kwargs)
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time.time()
        print('total time:', self.end -self.start)

In [50]:
with ExecutionTimeCalculator(time.sleep, 1) as etc:
    pass

total time: 1.0010366439819336


In [51]:
@contextmanager
def execute_time_calculate(func, *args, **kwargs):
    try:
        start = time.time()
        yield func(*args, **kwargs)
    finally:
        end = time.time()
        print("total time:", end - start)
        

In [55]:
with execute_time_calculate(time.sleep, 1) as etc:
    pass

total time: 1.0011117458343506


## WHY

**Key Takeaways**

- The with statement simplifies exception handling by encapsu-lating standard uses of try/finally statements in so-called context managers. 

- Most commonly it is used to manage the safe acquisition and release of system resources. Resources are acquired by the with statement and released automatically when execution leaves the with context. 

- Using with effectively can help you avoid resource leaks and make your code easier to read. 